In [41]:
import pandas as pd
import datetime

# STEP 1: Load variant file
variant_df = pd.read_excel("varient.xlsx")
variant_df = variant_df[variant_df['Variant'] == 1]

# STEP 2: Load basket file with multiple tabs
xls = pd.ExcelFile("staging_data_all.xlsx")
all_data = []

for sheet in xls.sheet_names:
    df = xls.parse(sheet)
    df['Tab'] = sheet  # Add tab/fruit name
    all_data.append(df)

# Combine all tabs
basket_df = pd.concat(all_data, ignore_index=True)

# STEP 3: Preprocessing
basket_df['Date'] = pd.to_datetime(basket_df['Date'])
basket_df['Year'] = basket_df['Date'].dt.year

# Extract hour from Time column (safely)
basket_df['Hour'] = basket_df['Time'].apply(
    lambda t: t.hour if isinstance(t, datetime.time) else pd.to_datetime(t).hour
)

# STEP 4: Merge with variant_df
merged_df = pd.merge(
    basket_df,
    variant_df[['Fruit', 'Color', 'Year']],
    left_on=['Tab', 'Year'],
    right_on=['Fruit', 'Year'],
    how='left',
    suffixes=('', '_variant')
)

# Sort for aggregation
merged_df = merged_df.sort_values(by=['Tab', 'Date', 'Hour']).reset_index(drop=True)

# STEP 5: Aggregate logic
result = []
skip_next = False

for i in range(len(merged_df) - 1):
    if skip_next:
        skip_next = False
        continue

    curr = merged_df.iloc[i]
    nxt = merged_df.iloc[i + 1]

    same_group = (
        curr["Tab"] == nxt["Tab"] and
        curr["Date"] == nxt["Date"] and
        curr["Hour"] == nxt["Hour"]
    )

    if same_group:
        allowed_colors = [c.strip() for c in str(curr['Color_variant']).split(',')]

        curr_color = curr["Color"]
        next_color = nxt["Color"]

        # Check both colors in variant color list and are different
        if (
            curr_color != next_color and
            curr_color in allowed_colors and
            next_color in allowed_colors
        ):
            new_row = curr.copy()
            new_row["Color"] = " + ".join(sorted(set([curr_color, next_color])))
            new_row["Weight"] = curr["Weight"] + nxt["Weight"]
            new_row["Sourness"] = curr["Sourness"] + nxt["Sourness"]
            new_row["Hour"] = curr["Hour"]  # Keep hour same, not summed
            result.append(new_row)
            skip_next = True
        else:
            result.append(curr)
    else:
        result.append(curr)

# Add final row if not merged
if not skip_next:
    result.append(merged_df.iloc[-1])

# Final output
final_df = pd.DataFrame(result)

# Display or save
final_df
# final_df.to_excel("final_aggregated_output.xlsx", index=False)


,Date,Time,Season,Size,Color,Country,Weight,Sourness,Tab,Year,Hour,Fruit,Color_variant
0,2024-05-01,17:00:00,Spring,Large,Green + Red,USA,2.0,5,Apple,2024,17,Apple,"Red, Green"
2,2024-06-10,17:00:00,Spring,Large,Yellow,USA,1.0,4,Lime,2024,17,NaN,NaN
3,2024-06-11,17:00:00,Spring,Large,Yellow,USA,1.0,4,Lime,2024,17,NaN,NaN


In [43]:
variant_df

,Fruit,Color,Year,Variant
1,Apple,"Red, Green",2024,1
2,Mango,"Yellow, Red",2023,1


In [45]:
basket_df.head()

,Date,Time,Season,Size,Color,Country,Weight,Sourness,Tab,Year,Hour
0,2024-05-01,17:00:00,Spring,Large,Red,USA,1.2,3,Apple,2024,17
1,2024-05-01,17:00:00,Spring,Small,Green,USA,0.8,2,Apple,2024,17
2,2024-06-10,17:00:00,Spring,Large,Yellow,USA,1.0,4,Lime,2024,17
3,2024-06-11,17:00:00,Spring,Large,Yellow,USA,1.0,4,Lime,2024,17


In [52]:
# Group by Tab and calculate the required metrics
result = final_df.groupby("Tab").apply(
    lambda g: pd.Series({
        "Total_Sourness": g["Sourness"].sum(),
        "Weighted_Avg_Weight": (g["Weight"] * g["Sourness"]).sum() / g["Sourness"].sum(),
        "Weighted_Avg_Hr": (g["Hour"] * g["Sourness"]).sum() / g["Sourness"].sum()

    })
).reset_index()

result


/var/folders/z1/vb6yvwyd2y14rfgw3h1909040000gn/T/ipykernel_19145/2151771158.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  result = final_df.groupby("Tab").apply(


,Tab,Total_Sourness,Weighted_Avg_Weight,Weighted_Avg_Hr
0,Apple,5.0,2.0,17.0
1,Lime,8.0,1.0,17.0
